In [1]:
from src.v1.db.duckdb_dataset import DuckDBDataset, DuckDBConfig

In [2]:
cfg = DuckDBConfig(threads=2, memory_limit="4GB", temp_directory="/tmp/duckdb_spill")
ds = DuckDBDataset("../data/epc_partitions/certificates", hive_partitioning=True, config=cfg, dataset_name="certs")


In [3]:
ds.schema()

[{'name': 'LMK_KEY', 'type': 'VARCHAR'},
 {'name': 'ADDRESS1', 'type': 'VARCHAR'},
 {'name': 'ADDRESS2', 'type': 'VARCHAR'},
 {'name': 'ADDRESS3', 'type': 'VARCHAR'},
 {'name': 'POSTCODE', 'type': 'VARCHAR'},
 {'name': 'BUILDING_REFERENCE_NUMBER', 'type': 'VARCHAR'},
 {'name': 'CURRENT_ENERGY_RATING', 'type': 'VARCHAR'},
 {'name': 'POTENTIAL_ENERGY_RATING', 'type': 'VARCHAR'},
 {'name': 'CURRENT_ENERGY_EFFICIENCY', 'type': 'BIGINT'},
 {'name': 'POTENTIAL_ENERGY_EFFICIENCY', 'type': 'BIGINT'},
 {'name': 'PROPERTY_TYPE', 'type': 'VARCHAR'},
 {'name': 'BUILT_FORM', 'type': 'VARCHAR'},
 {'name': 'INSPECTION_DATE', 'type': 'DATE'},
 {'name': 'LOCAL_AUTHORITY', 'type': 'VARCHAR'},
 {'name': 'CONSTITUENCY', 'type': 'VARCHAR'},
 {'name': 'COUNTY', 'type': 'VARCHAR'},
 {'name': 'LODGEMENT_DATE', 'type': 'DATE'},
 {'name': 'TRANSACTION_TYPE', 'type': 'VARCHAR'},
 {'name': 'ENVIRONMENT_IMPACT_CURRENT', 'type': 'BIGINT'},
 {'name': 'ENVIRONMENT_IMPACT_POTENTIAL', 'type': 'BIGINT'},
 {'name': 'ENER

In [6]:
# 2) Get a small preview
df = ds.head(10, columns=["LMK_KEY","postcode_norm","outcode2","lodgement_year"])
df

,LMK_KEY,postcode_norm,outcode2,lodgement_year
0,19183c9849256006c1bfb13852490504298759a0755e48...,SF40FY,SF,2022
1,29369a844cf9e99d8b18b6cfe490643bb9aecb99b6292f...,SF40FY,SF,2022
2,0750f199a68a1480e06d50011377b25ba833fe9a80c0e5...,SF40FY,SF,2022
3,82583199202018121319303745589728,SA409QD,SA,2018
4,963923539962018081816074480788788,SA311HE,SA,2018
5,748864849142018062117133491582398,SA58DT,SA,2018
6,1644928918832018082221112448278708,SA91BD,SA,2018
7,1490463872832018020110590565078906,SA48TF,SA,2018
8,260726329002018052318401865082128,SA699NL,SA,2018
9,1655288339962018081317145279078108,SA16JE,SA,2018


In [7]:
# 3) Filter + select (pushdown happens inside DuckDB)
df = ds.select(
    columns=["LMK_KEY","property_type","built_form","lodgement_date"],
    where={"outcode2":"GL", "lodgement_year__gte":"2019"},
    limit=1000,
    to="pandas",   # or "arrow"
)
df

,LMK_KEY,PROPERTY_TYPE,BUILT_FORM,LODGEMENT_DATE
0,286711123712020031218553420900964,Flat,Mid-Terrace,2020-03-12
1,152823822802020011716253553109638,Flat,Enclosed Mid-Terrace,2020-01-17
2,1183361022702020072116592623602498,House,Semi-Detached,2020-07-21
3,1802975742962020061510254280728450,House,Semi-Detached,2020-06-15
4,62807632242020011623294744409558,House,Mid-Terrace,2020-01-16
...,...,...,...,...
995,1778507362262020012418225858898600,Flat,End-Terrace,2020-01-24
996,1795663702222020033112182929238870,House,Semi-Detached,2020-03-31
997,215078757012020070410372227000859,House,Semi-Detached,2020-07-04
998,389207646512020081012094022000069,House,Mid-Terrace,2020-08-10


In [9]:
# 4) Distinct partition values
parts = ds.distinct_values(["outcode2","lodgement_year"], limit=500)
parts

,outcode2,lodgement_year
0,TM,2022
1,TD,2025
2,TD,2015
3,TD,2012
4,TD,2019
...,...,...
495,DN,2009
496,FY,2010
497,DN,2022
498,DN,2013


In [10]:
# 5) Value counts
vc = ds.value_counts("property_type", where={"outcode2":"SW"}, limit=20)
vc

,value,n
0,Flat,391319
1,House,124299
2,Maisonette,43429
3,Bungalow,1573
4,Park home,1


In [12]:
# 6) Raw SQL against a view (self.view_name)
#    You can reference the materialized view directly:
view = ds.view_name
df_sql = ds.sql(f"""
    SELECT outcode2, lodgement_year, COUNT(*) AS n
    FROM {view}
    WHERE outcode2 IN ('GL','SW') AND lodgement_year >= '2018'
    GROUP BY outcode2, lodgement_year
    ORDER BY outcode2, lodgement_year
""")
df_sql

,outcode2,lodgement_year,n
0,GL,2018,15241
1,GL,2019,18180
2,GL,2020,17995
3,GL,2021,18538
4,GL,2022,19574
5,GL,2023,17972
6,GL,2024,18077
7,GL,2025,10311
8,SW,2018,26627
9,SW,2019,34891
